In [16]:
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine
from config import pg_password

In [17]:
ticker_csv = pd.read_csv('ticker.csv')
ticker_daily = pd.read_csv('ticker_daily.csv')
ticker_sentiment = pd.read_csv('sentiment.csv')

In [18]:
ticker_csv_df = pd.DataFrame(ticker_csv)
ticker_csv_df.drop(columns=['Unnamed: 0'],inplace=True)
ticker_csv_df.head()

,symbol,company_name
0,ACIW,ACI Worldwide
1,ACLS,Axcelis Technologies
2,COHU,Cohu
3,MDLA,Medallia
4,MSFT,Microsoft


In [19]:
ticker_daily_df = pd.DataFrame(ticker_daily)
ticker_daily_df.head()

,ticker,opening,closing,high,low,date
0,ACIW,38.68,37.88,38.69,37.18,2021-01-04
1,ACIW,37.75,38.00,38.18,37.74,2021-01-05
2,ACIW,38.18,38.63,39.32,37.96,2021-01-06
3,ACIW,38.98,39.14,39.19,38.70,2021-01-07
4,ACIW,39.50,39.21,40.29,38.93,2021-01-08


In [20]:
Ticker_agg = ticker_daily_df.groupby(['ticker']).agg({'opening': np.mean, 'closing':np.mean, 'high':np.max, 'low':np.min}).round(2)

In [21]:
Ticker_agg.rename(columns={"opening": "avg_opening", "closing": "avg_closing", "high":"max_high", "low": "min_low"}, inplace=True)
Ticker_agg

,avg_opening,avg_closing,max_high,min_low
ticker,,,,
ACIW,39.43,39.38,43.23,35.59
ACLS,39.34,39.34,50.21,28.45
COHU,42.68,42.51,51.86,33.40
CRM,225.05,224.80,251.23,201.51
MDLA,32.95,32.88,48.28,23.58
MSFT,240.82,241.02,263.19,211.94
NOW,516.11,514.87,598.37,448.27
PANW,355.36,354.49,403.00,311.56
PAR,73.02,72.91,90.35,57.46


In [22]:
ticker_name = pd.merge(ticker_csv_df,Ticker_agg, left_on='symbol',right_on='ticker')
ticker_name.rename(columns={'symbol':'ticker'},inplace=True)
ticker_name

,ticker,company_name,avg_opening,avg_closing,max_high,min_low
0,ACIW,ACI Worldwide,39.43,39.38,43.23,35.59
1,ACLS,Axcelis Technologies,39.34,39.34,50.21,28.45
2,COHU,Cohu,42.68,42.51,51.86,33.40
3,MDLA,Medallia,32.95,32.88,48.28,23.58
4,MSFT,Microsoft,240.82,241.02,263.19,211.94
5,PANW,Palo Alto Networks,355.36,354.49,403.00,311.56
6,PAR,PAR Technology,73.02,72.91,90.35,57.46
7,PYPL,PayPal Holdings,255.85,255.56,309.14,223.09
8,CRM,Salesforce.com,225.05,224.80,251.23,201.51
9,NOW,ServiceNow,516.11,514.87,598.37,448.27


In [23]:
ticker_sentiment_df = pd.DataFrame(ticker_sentiment)
ticker_sentiment_df.drop(columns=['Unnamed: 0'],inplace=True)
ticker_sentiment_df.head()

,ticker,date,news_title,compound_score
0,ACIW,2021-06-12,"Is ACI Worldwide, Inc. (NASDAQ:ACIW) Trading A...",0.000
1,ACIW,2021-06-08,Indian Consumers More Concerned About Digital ...,-0.586
2,ACIW,2021-06-01,ACI Worldwide Powers Payments Innovation for P...,0.296
3,ACIW,2021-05-28,ACI Worldwide to Attend Craig-Hallum Instituti...,0.000
4,ACIW,2021-05-25,ACI Worldwide and J.P. Morgan Collaborate to O...,0.361


In [24]:
password = pg_password
connection_string = f"postgres:{password}@localhost:5432/stocks"
engine = create_engine(f'postgresql://{connection_string}')

In [25]:
ticker_name.to_sql(name='ticker_name', con=engine, if_exists='append', index=False)

In [26]:
ticker_daily_df.to_sql(name='ticker_daily', con=engine, if_exists='append', index=True)

In [27]:
ticker_sentiment_df.to_sql(name='sentiment', con=engine, if_exists='append', index=True)